## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [1]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [2]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Reading Datasets

In [3]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [4]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [5]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.573333,0.018448,0.004667,0.003236,0.006212,0.003818,SmoothSubspace,ACF&PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF&PACF
1,1,0.566667,0.012903,0.005363,0.001630,0.001871,0.003664,SmoothSubspace,ACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,ACF
2,2,0.613333,0.019624,0.006083,0.003176,0.005806,0.004071,SmoothSubspace,PACF: n_random_points=1 nb_inst_per_class=1,1,1,Rsast,PACF


In [6]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['SmoothSubspace', 'SonyAIBORobotSurface2',
       'DistalPhalanxOutlineCorrect', 'GunPoint', 'ItalyPowerDemand',
       'DistalPhalanxOutlineAgeGroup', 'ShapeletSim', 'SyntheticControl',
       'SonyAIBORobotSurface1', 'Chinatown', 'Coffee', 'BirdChicken'],
      dtype=object)

In [7]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [8]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="STC"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.776667,0.830000,0.713333,0.755333,0.708667,0.837333,BirdChicken,0.950000
1,Chinatown,0.950865,0.962177,0.938503,0.960233,0.947833,0.963460,Chinatown,0.967930
2,Coffee,0.937143,0.997143,0.951429,0.937143,0.994286,0.992857,Coffee,1.000000
3,DistalPhalanxOutlineAgeGroup,0.613813,0.641823,0.559808,0.551367,0.543789,0.638082,DistalPhalanxOutlineAgeGroup,0.769784
4,DistalPhalanxOutlineCorrect,0.450097,0.433671,0.476039,0.452029,0.472850,0.454976,DistalPhalanxOutlineCorrect,0.771739
5,GunPoint,0.790722,0.973167,0.798056,0.955222,0.929111,0.967444,GunPoint,0.993333
6,ItalyPowerDemand,0.908698,0.917080,0.902098,0.899992,0.908107,0.913443,ItalyPowerDemand,0.955296
7,ShapeletSim,0.944296,0.891037,0.895704,0.734593,0.631259,0.881037,ShapeletSim,1.000000
8,SmoothSubspace,0.823500,0.868444,0.823556,0.857500,0.829389,0.869722,SmoothSubspace,0.933333
9,SonyAIBORobotSurface1,0.742696,0.789063,0.743738,0.722374,0.739567,0.789040,SonyAIBORobotSurface1,0.752080


In [9]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.835254,0.686961,0.755086,0.765462,0.749563,0.762248
ACF&PACF,0.922074,0.765941,0.789719,0.798188,0.791289,0.798655
Max ACF,0.835801,0.675217,0.738975,0.747021,0.722621,0.753915
Max PACF,0.908536,0.693922,0.760655,0.762988,0.744161,0.761031
None,0.884608,0.692410,0.739369,0.739672,0.729103,0.737141
PACF,0.920374,0.769255,0.783490,0.803086,0.800852,0.796269


In [10]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.835201,0.681524,0.740775,0.769928,0.759195,0.767933
ACF&PACF,0.924836,0.756769,0.778422,0.804519,0.800027,0.802234
Max ACF,0.834418,0.641934,0.739348,0.753301,0.744185,0.759893
Max PACF,0.912417,0.677018,0.762932,0.763736,0.752552,0.763962
None,0.889856,0.660201,0.737255,0.752651,0.735296,0.748832
PACF,0.923814,0.760062,0.780487,0.800277,0.807792,0.802065


#### Time: Subsequence Lenght Method 

In [11]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,,
ACF,81.630738,0.143859,29.515313,132.824072,177.888586,42.810175,0.674857,7.645123,1.599955,0.419210,0.815245,128.598058
ACF&PACF,297.280662,0.444094,102.533573,419.191777,519.712858,95.588660,1.649303,222.211859,2.648634,2.247746,3.008572,402.314136
Max ACF,2.745482,0.145839,2.160797,49.805655,59.238734,4.674037,0.681721,2.640422,1.595359,0.402863,0.627533,45.391469
Max PACF,107.885490,0.162721,6.330146,29.798286,35.902560,4.200933,0.747006,124.217464,1.849311,0.405555,0.495278,29.918500
None,1.853694,0.133453,1.236922,28.495936,34.720866,2.811864,0.665972,1.836391,1.571106,0.300002,0.397229,28.232846
PACF,208.928276,0.387296,74.365190,293.491555,345.791509,52.941237,1.651981,215.296044,2.606331,1.952652,2.348842,260.544206


In [12]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,10.101484,2.387529,23.632669,100.484380,49.611188,69.598278
ACF&PACF,22.398413,9.917575,94.245407,324.086783,175.067904,231.727303
Max ACF,1.604542,0.585716,4.991716,31.109821,13.325211,20.310050
Max PACF,1.482923,2.655167,25.899580,41.185337,34.904446,35.340083
None,1.073879,0.412985,3.146364,18.552939,8.169355,12.155648
PACF,12.859020,7.130818,70.826133,224.675135,122.197625,162.759408


In [13]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,15.471265,1.431632,12.940310,126.186758,38.197201,63.417628
ACF&PACF,34.245723,22.613324,56.727326,397.237630,134.331238,216.324041
Max ACF,1.987199,0.484151,3.796596,36.510744,10.960785,18.317938
Max PACF,1.663263,18.329124,19.866450,42.681651,27.178506,31.809976
None,1.249692,0.307491,2.311166,22.192451,6.672976,10.837288
PACF,19.179620,21.065382,43.419565,272.502505,98.214997,148.219240


#### Variance: Subsequence Lenght Method 

In [14]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.006834,0.041080,0.047908,0.041822,0.042981,0.042456
ACF&PACF,0.001681,0.044881,0.051074,0.044639,0.045830,0.047244
Max ACF,0.007137,0.049319,0.047022,0.047410,0.050501,0.044524
Max PACF,0.002512,0.058850,0.049142,0.052957,0.056568,0.053708
None,0.004699,0.053859,0.053265,0.050817,0.056449,0.050489
PACF,0.001609,0.040613,0.050537,0.046929,0.040042,0.045886


In [15]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10,100,30,50
len_method,,,,,,
ACF,0.006816,0.050875,0.043880,0.038861,0.043958,0.039875
ACF&PACF,0.002764,0.051615,0.046656,0.044399,0.046758,0.045000
Max ACF,0.007106,0.051093,0.047559,0.043758,0.058082,0.043624
Max PACF,0.003822,0.063731,0.049914,0.049487,0.060512,0.049859
None,0.006006,0.061807,0.051618,0.048585,0.055942,0.051358
PACF,0.002436,0.045380,0.051070,0.040347,0.042670,0.045357


#### Another Results

In [16]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,6180,6180,6180
unique,12,6,216
top,SmoothSubspace,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,720,1030,60


In [17]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.029526
Coffee,1.500986
DistalPhalanxOutlineAgeGroup,19.866818
DistalPhalanxOutlineCorrect,24.442815
GunPoint,6.767564
ItalyPowerDemand,0.202361
ShapeletSim,11.955152
SmoothSubspace,0.395690


In [18]:
df_result['time'].sum()/(60*60)

98.77257662660546

#### Generate CD Diagram Tunning Hyperparameter

In [19]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()
df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')].head(5)

,classifier_name,dataset_name,accuracy
0,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,GunPoint,0.979556
1,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,ItalyPowerDemand,0.945578
2,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,SmoothSubspace,0.888444
3,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,SonyAIBORobotSurface2,0.898496
4,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,GunPoint,0.955556


In [20]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not 

In [ ]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

In [ ]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

#### Focus on most accurate lenght method

##### Overall Accuracy

In [ ]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

In [ ]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

##### Overall time complexity

In [ ]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

In [ ]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')